### Getting data from binance with its api

Python library.
https://github.com/sammchardy/python-binance

Code based on
https://medium.com/swlh/retrieving-full-historical-data-for-every-cryptocurrency-on-binance-bitmex-using-the-python-apis-27b47fd8137f

#### Import libraries

In [2]:
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
import json

#### Binance keys

In [3]:
## Load credentials from json file
cr = open('cred.json',) 
data = json.load(cr)  

In [7]:
### CONSTANTS

binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750

## Client
binance_client = Client(api_key=data["binance_api_key"], api_secret=data["binance_api_secret"])

#### Functions

In [8]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  
        old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": 
        old = datetime.strptime("1 May 2020", "%d %b %Y")
    if source == "binance": 
        new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit="ms") 
    return old, new



def get_all_binance(symbol, kline_size, save = False):
    
    filename = "crypto_data/%s-%s-data.csv" % (symbol, kline_size)
    
    ## If file exists 
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
    else: 
        data_df = pd.DataFrame()
        
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    
    delta_min = (newest_point - oldest_point).total_seconds()/60
    
    available_data = math.ceil(delta_min/binsizes[kline_size])
    
    if oldest_point == datetime.strptime("1 May 2020", "%d %b %Y"): 
        print("Downloading all available %s data for %s. Be patient..!" % (kline_size, symbol))
        
    else: 
        print("Downloading %d minutes of new data available for %s, i.e. %d instances of %s data." % (delta_min, symbol, available_data, kline_size))
        
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    
    data = pd.DataFrame(klines, columns = ["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_av", "trades", "tb_base_av", "tb_quote_av", "ignore"])
    
    data["timestamp"] = pd.to_datetime(data["timestamp"], unit="ms")
    
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
        
    data_df.set_index("timestamp", inplace=True)
    
    if save: 
        data_df.to_csv(filename)
        
    print("All caught up..!")
    
    return data_df

#### Getting data

In [13]:
""" 
binance_symbols = ['ZECBUSD',
 'NEOBUSD',
 'ETCBUSD',
 'LTCBUSD',
 'EOSBUSD',
 'FLMBUSD',
 'YFIIBUSD',
 'BZRXBUSD',
 'BANDBUSD',
 'GRTBUSD',
 'ZRXBUSD',
 'RSRBUSD',
 'SXPBUSD',
 'LINKBUSD',
 'ETHBUSD',
 'TWTBUSD',
 'ZILBUSD',
 'TRUBUSD',
 'AAVEBUSD',
 'BCHBUSD',
 'DODOBUSD',
 'OMGBUSD',
 'LRCBUSD',
 'BTTBUSD',
 'RUNEBUSD',
 'YFIBUSD',
 'NANOBUSD',
 'KSMBUSD',
 'ALGOBUSD']
 
""" 
binance_symbols = ["NEOBUSD","LINKBUSD"]

for symbol in binance_symbols:
    get_all_binance(symbol, '1m', save = True)

All caught up..!
All caught up..!
